In [1]:
from Bio import AlignIO, SeqIO, Seq, pairwise2
from Bio.pairwise2 import format_alignment
import pandas as pd
import os
import numpy as np

/home/norbert/miniconda3/lib/python3.11/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
min_length_aligned_sequence = 30 #Minimal lenght of sequence which could be an exon
extreme_homology = 0.97 #percentage of homology of sequence, treshold #I assume two faulty aligned nucleotides per 100 (98%) and one more nt because sometimes latest nt can move from end of one sequence to beginning next sequence
middle_homology = 0.9 #used for classification of internal exons

In [9]:
def cutting_scrap_right(directory, acceptable_gap_length):
    data_frames = []
    for filename in os.listdir(directory):
        file = os.path.join(directory, filename)
        if not os.path.isfile(file):
            continue
        if file.endswith(".aln"):
            gaps_signs = "-" * acceptable_gap_length #maximum length of gaps in sequence in one exon's sequence
    
            alignment = AlignIO.read(file, "clustal")
    
        
        
            while True:
                first_nucleotides_pair = alignment[:, 0]#first nucleotides in both strands as variable
                if "-" in first_nucleotides_pair: #deleting mismatches at the start 
                    alignment = alignment[:, 1:]
                    #print(f"alignment after cut:\n{alignment}") #chekpoint
                    continue
    
            
        
                #changing extension
                seq1 = alignment[0].seq #transcript sequence
                seq2 = alignment[1].seq #gene sequence
        
                
                #Counting distance:
                first_gaps_position_seq1 = seq1.find(gaps_signs) #first gap inddex.
                first_gaps_position_seq2 = seq2.find(gaps_signs) 
                #print(f"first gap position in first sequence:{first_gaps_position_seq1}, in 2nd sequence: {first_gaps_position_seq2}") #checkpoint
                
                if first_gaps_position_seq2 < 0: #avoiding faults when find() function returns "-1" as finding nothing
                    distance_between_first_nt_and_gap = first_gaps_position_seq1
                else:
                    distance_between_first_nt_and_gap = min(first_gaps_position_seq1, first_gaps_position_seq2)
        
                
                local_alignment = pairwise2.align.localxx(seq1[:distance_between_first_nt_and_gap], seq2[:distance_between_first_nt_and_gap], \
                                                            one_alignment_only = True)
                #"xx"  means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment to count homology
                #print("local_alignment:", local_alignment) #checkpoint
               
                try: 
                    local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short, sequence has not exons or other fault. \n")
                        break
                    continue
                
                
                if local_homology_percentage <= extreme_homology:
                    #print(f"\n Too low score of homology. \n alignment: {alignment[:, :50]} \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, percent: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, distance_between_first_nt_and_gap:] 
                    
                else:
                    if distance_between_first_nt_and_gap <= min_length_aligned_sequence:
                        #print(f"\n alignment is too short. \n distance between sequences {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, procentowo: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, distance_between_first_nt_and_gap:]
                    else:
                        #print(f"Sequence {alignment[0].id} cut properly from LEFT side, \n distance between sequences: {distance_between_first_nt_and_gap}")
                        #alignment: {alignment[:, :50]},
                        break
                
        
            
        ######################### TERAZ OD PRAWEJ DO LEWEJ #######################  
        
            
            while True:
                last_nucleotides_pair = alignment[:, -1]#last nucleotides in both strands as variable
                if "-" in last_nucleotides_pair: #deleting mismatches at the end
                    alignment = alignment[:, :-1]
                    #print(f"alignment check:\n{alignment[:, -50:-1]}") checkpoint
                    continue
                        
                            
                seq1 = alignment[0].seq #transcript sequence
                seq2 = alignment[1].seq #gene sequence
                
                last_gaps_position_seq1 = seq1.rfind(gaps_signs)  #Counting distance: #getting index of first gap. 
                last_gaps_position_seq1 = last_gaps_position_seq1 + acceptable_gap_length
                
                last_gaps_position_seq2 = seq2.rfind(gaps_signs)  
                last_gaps_position_seq2 = last_gaps_position_seq2 + acceptable_gap_length #now we have to add length of gap to index, because python starts counting from 0 nt, and we just change direction of counting.
                #print("seq1:", last_gaps_position_seq1, "seq2:", last_gaps_position_seq2) checkpoint
                
                
                if last_gaps_position_seq2 < 0: #it means, if there is no gaps in second sequention, gap position is gap position in seq1. Additional, rfind() function return "-1" if it won't find any gaps in seq
                    distance_between_first_nt_and_gap = (len(seq1) - last_gaps_position_seq1)
                    distance_between_first_nt_and_gap = distance_between_first_nt_and_gap - 1 #correction to counting from 0th index
                    #print("pierwszy", distance_between_first_nt_and_gap)
                else:
                    distance_between_first_nt_and_gap = (len(seq1) - max(last_gaps_position_seq1, last_gaps_position_seq2))
                    #print("drugi", distance_between_first_nt_and_gap)
                    
                    
                    local_alignment = pairwise2.align.localxx(seq1[-distance_between_first_nt_and_gap:], seq2[-distance_between_first_nt_and_gap:], \
                                                            one_alignment_only = True)
                    #"xx" means no gap penalty while opening gaps or longering them and no penalties for mismatch. Just pure score of alignment
                #print("local_alignment:", local_alignment) #checkpoint
                
                
        
                try:
                    local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) 
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue
        
                
                if local_homology_percentage < extreme_homology:
                    #print(f"\n Too low homology! \n alignment: {alignment[:, -50:]} \n score: {local_alignment[0].score}, \distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                    alignment = alignment[:, :-distance_between_first_nt_and_gap] 
                    
                else:
                    if distance_between_first_nt_and_gap < min_length_aligned_sequence:
                        #print(f"\n Alignment is too short!. \n distance: {distance_between_first_nt_and_gap}, \n alignment: {alignment}, \n score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, %%%: {round(local_homology_percentage, 2)*100}%, \n {local_alignment}")
                        alignment = alignment[:, :-distance_between_first_nt_and_gap]
                    else:
                        #print(f"Sequence: {alignment[0].id} cut properly from RIGHT side")
                        #print(f"""\n \n \n victory prawo! \n distance between sequences {distance_between_first_nt_and_gap},
                        #alignment: {alignment[:, -50:]}, 
                        #score: {local_alignment[0].score}, distance: {distance_between_first_nt_and_gap}, 
                        #percent: {round(local_homology_percentage, 4)*100}%, \n \n local alignment to confirm: \n{local_alignment}""")
                        break
        
        
        
            
            nucleotides = ["a", "t", "g", "c", "A", "T", "G", "C"]
            index = 0
            id = []
            is_it_intron = []
            class_of_exon = []
            exon_or_intron_number = []
            percent_of_homology_for_exons = []
            length = []
            first_10_nt = []
            last_10_nt = []
            sequence = []
            path = []
            
            while True:
                try: 
                    local_homology_percentage = (local_alignment[0].score / distance_between_first_nt_and_gap) #that sign "[]" becouse local_alignment's type is list
                except IndexError:
                    if len(seq1) < 100:
                        print(f"\n \n {alignment[0].id} sequence is too short or sequence has not exons. \n")
                        break
                    continue
                
                if len(seq1) > 1:
                        
                    if seq1[0] != gaps_signs: #mining data from exons   
                        index += 1
                        last_nt_exon_index = seq1.find(gaps_signs) 
                
                        if last_nt_exon_index == -1: #it means if it is last exon, if find() don't find any gap signs at the end
                            last_nt_exon_index = len(seq1)
                                
                        alignment_of_exon = pairwise2.align.localxx(seq1[:last_nt_exon_index], seq2[:last_nt_exon_index], one_alignment_only = True)
                        #print( "\n alignment since start to the end of exon:", alignment_of_exon[0])
                        percent_of_homology = alignment_of_exon[0].score / last_nt_exon_index
                          
                        #print("exon analising finished")
                        #print(f"score alignmentu = , {alignment_of_exon[0].score}\n seq1 = {seq1}, \n alignment = , {alignment_of_exon[0]}")
                            
            

                    id.append(alignment[0].id)
                    exon_or_intron_number.append(index)
                    is_it_intron.append(False)
                    percent_of_homology_for_exons.append(round(percent_of_homology * 100, 2))
                    length.append(last_nt_exon_index)
                    first_10_nt.append(str(seq1[:10]))
                    last_10_nt.append(str(seq1[last_nt_exon_index-10 : last_nt_exon_index])) 
                    sequence.append(str(seq1[:last_nt_exon_index]))
                    path.append(str(file))
                            
                    if percent_of_homology >= 0.9 and last_nt_exon_index >= min_length_aligned_sequence :
                        class_of_exon.append("1")
                    elif percent_of_homology < 0.9 and percent_of_homology > 0 and last_nt_exon_index < min_length_aligned_sequence:
                        class_of_exon.append("2")
                    else:
                        class_of_exon.append("3")
            
                    seq1 = seq1[last_nt_exon_index:]
                    seq2 = seq2[last_nt_exon_index:]
                    #print("analising exon finished")
            
                            
            
                if len(seq1) > 1 and seq1[0] == "-":
                    index += 1
            
                    for i, nukleotide in enumerate(seq1):
                        if nukleotide not in nucleotides:
                            last_nt_intron_index = i+1
                        else:
                            break
                            
                    id.append(alignment[0].id)
                    exon_or_intron_number.append(index)
                    is_it_intron.append(True) 
                    percent_of_homology_for_exons.append(None)
                    length.append(last_nt_intron_index)
                    first_10_nt.append(str(seq2[:10]))
                    last_10_nt.append(str(seq2[last_nt_intron_index-10 : last_nt_intron_index]))
                    sequence.append(str(seq2[:last_nt_intron_index]))
                    path.append(str(file))
                    class_of_exon.append(None)
        
                    seq1 = seq1[last_nt_intron_index:]
                    seq2 = seq2[last_nt_intron_index:]
                    
                else:
                    break
                        
            dictionary = {"ID":id,
                          "exon_or_intron_number":exon_or_intron_number, 
                          "is_it_intron":is_it_intron, 
                          "class_of_exon":class_of_exon, 
                          "percent_of_homology_for_exons":percent_of_homology_for_exons, 
                          "length":length,
                          "first_10_nt" : first_10_nt,
                          "last_10_nt" : last_10_nt, 
                          "path":path, 
                          "sequence" : sequence}
            data_frame = pd.DataFrame(dictionary)
            #print(data_frame)
        data_frames.append(data_frame)
        #print(data_frames)


    final_data_frame = pd.concat(data_frames, ignore_index=True)
    final_data_frame.index = np.arange(1, len(final_data_frame) + 1)
    print(final_data_frame)

    def save_to_file(final_data_frame, filename):
        if os.path.isfile(filename):
            user_input = input("File already exists. Do you want overwrite? y/n: ")
            if user_input == "n":
                base_name, ext = os.path.splitext(filename)
                i = 1
                while os.path.isfile(f"{base_name}_{i}{ext}"):
                    i += 1
                filename = f"{base_name}_{i}{ext}"
                final_data_frame.to_csv(filename, sep = "\t")
                
            elif user_input == "y":
                final_data_frame.to_csv(filename, sep = "\t")
            else:
                print("Clarify your answer")
                
        else:
            final_data_frame.to_csv(filename, sep = "\t")
            

    save_to_file(final_data_frame, "przyciete_introny.tsv")

cutting_scrap_right('./alingment_gen_transkrypt/', 2)


#zapisac w formacie gff
 #cele na teraz:

#mamy dwie opcje, albo ręcznie ustawiamy to dla gff przez np. tworzenie list, albo szukamy informacji jak obecne dane i ich rozszerzenia mogą posłużyć do konwertowania na formaty bardziej przyjazne gff. Ma to większy próg wejścia ale więcej uczy i minimalizuje błędy









 
 EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05 sequence is too short, sequence has not exons or other fault. 


 
 EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05 sequence is too short or sequence has not exons. 


 
 EL_OG0030308_t2_g2_EL_PAM1_GP153_OV05 sequence is too short or sequence has not exons. 

                                        ID  exon_or_intron_number  \
1    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    1.0   
2    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    2.0   
3    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    3.0   
4    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    4.0   
5    EG_OG0030307_t1_g1_EG_PAM1_GP153_OV05                    5.0   
..                                     ...                    ...   
144  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                   11.0   
145  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                   12.0   
146  EG_OG0030309_t3_g3_EG_PAM1_GP153_OV05                   13.0   
147  EG_OG0030309_

File already exists. Do you want overwrite? y/n:  25


Clarify your answer
